In [1]:
import os
os.environ["SPARK_HOME"] = "/Applications/spark"
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_PYTHON"] = "python"

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark-ml") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .getOrCreate()

:: loading settings :: url = jar:file:/Applications/Spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/kavisanthoshkumar/.ivy2/cache
The jars for the packages stored in: /Users/kavisanthoshkumar/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d414dd83-a379-4b78-90f1-4ba8ecea3d7b;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 86ms :: artifacts dl 2ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	----------

In [3]:
import os
import pandas as pd
import numpy as np

import pyspark.sql.functions as F
from pyspark.sql.types import (
    StructType, 
    StructField, 
    IntegerType, 
    FloatType, 
    StringType
)
from pyspark.sql import DataFrame

In [4]:
data = spark.read.parquet("../dataset/month_partition_online_shoppers_intention", header= True, inferSchema = True)

data.show()

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+-----------+-----------+----------+----------------+-------+------+-----------+-----------------+-------+-------------+-------------------+-----+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|  ExitRates| PageValues|SpecialDay|OperatingSystems|Browser|Region|TrafficType|      VisitorType|Weekend|made_purchase|ingestion_timestamp|Month|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+-----------+-----------+----------+----------------+-------+------+-----------+-----------------+-------+-------------+-------------------+-----+
|             1|                   39.2|            2|                 120.8|             7|                   80.5|        0.0|       0.01|        0.0|       0.0|               

#### 1. Identify null counts for each column

In [5]:
col_name = data.columns

null_counts = data.select(
    [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in col_name]
)
null_counts.show()

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+----------------+-------+------+-----------+-----------+-------+-------------+-------------------+-----+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|ExitRates|PageValues|SpecialDay|OperatingSystems|Browser|Region|TrafficType|VisitorType|Weekend|made_purchase|ingestion_timestamp|Month|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+----------------+-------+------+-----------+-----------+-------+-------------+-------------------+-----+
|             0|                      0|            0|                     0|             0|                      0|          0|        0|         0|         0|               0|      0|     0|          0| 

#### 2. Fill null numeric values with column averages.

In [6]:
numeric_col_names = []
for col, col_type in data.dtypes:
    if col_type in ('int', 'double'):
        numeric_col_names.append(col)

print(f"length of numeric cols : {len(numeric_col_names)}")

length of numeric cols : 14


In [7]:
# Computing Average Value with respect to Each Column
avg_dict_values = data.select(
    [F.mean(c).alias(c) for c in numeric_col_names]
).collect()[0].asDict()


# If-Else Condition replacing null values with average value
conditions = [F.when(F.col(key).isNull(), val).otherwise(F.col(key)).alias(key) 
                    for key, val in avg_dict_values.items()]   


data.select(*conditions).show()

26/01/25 17:22:34 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+-----------+-----------+----------+----------------+-------+------+-----------+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|  ExitRates| PageValues|SpecialDay|OperatingSystems|Browser|Region|TrafficType|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+-----------+-----------+----------+----------------+-------+------+-----------+
|           1.0|                   39.2|          2.0|                 120.8|           7.0|                   80.5|        0.0|       0.01|        0.0|       0.0|             3.0|    2.0|   4.0|        2.0|
|           3.0|                   89.6|          0.0|                   0.0|          57.0|            1721.906667|        0.0|0.005932203|204.0079491|       0.0|     

In [8]:
# Alternate way - Filling null values with average
data = data.fillna(avg_dict_values)

#### 3. Trim leading/trailing spaces from string columns.

In [9]:
string_col_names = [col for col, col_dtype in data.dtypes if col_dtype == 'string']

for col in string_col_names:
    data = data.withColumn(col, F.trim(col))
    print(f"Trimmed Column: {col}")


data.show()

Trimmed Column: VisitorType
Trimmed Column: Month
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+-----------+-----------+----------+----------------+-------+------+-----------+-----------------+-------+-------------+-------------------+-----+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|  ExitRates| PageValues|SpecialDay|OperatingSystems|Browser|Region|TrafficType|      VisitorType|Weekend|made_purchase|ingestion_timestamp|Month|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+-----------+-----------+----------+----------------+-------+------+-----------+-----------------+-------+-------------+-------------------+-----+
|             1|                   39.2|            2|                 120.8|             7|                   80.5|        0.0|

#### 4. Validate that duration columns are non‑negative.

In [10]:
#### We don't see any columns have negative values ####
duration_cols = ['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration']

negVal_dict = data.select(
    *[F.count(F.when(F.col(col) < 0, col)).alias(f"NegCount_{col}") for col in duration_cols]
).collect()[0].asDict()


negVal_dict

{'NegCount_Administrative_Duration': 0,
 'NegCount_Informational_Duration': 0,
 'NegCount_ProductRelated_Duration': 0}

#### 5. Cast Weekend and made_purchase to boolean explicitly.

In [11]:
data = (data.withColumn('Weekend', F.col("Weekend").cast("boolean"))
    .withColumn("made_purchase", F.col("made_purchase").cast("boolean")))

data.printSchema()

root
 |-- Administrative: integer (nullable = true)
 |-- Administrative_Duration: double (nullable = false)
 |-- Informational: integer (nullable = true)
 |-- Informational_Duration: double (nullable = false)
 |-- ProductRelated: integer (nullable = true)
 |-- ProductRelated_Duration: double (nullable = false)
 |-- BounceRates: double (nullable = false)
 |-- ExitRates: double (nullable = false)
 |-- PageValues: double (nullable = false)
 |-- SpecialDay: double (nullable = false)
 |-- OperatingSystems: integer (nullable = true)
 |-- Browser: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- TrafficType: integer (nullable = true)
 |-- VisitorType: string (nullable = true)
 |-- Weekend: boolean (nullable = true)
 |-- made_purchase: boolean (nullable = true)
 |-- ingestion_timestamp: date (nullable = true)
 |-- Month: string (nullable = true)



#### 6. Remove duplicate rows if present.

In [12]:
try:
    assert(data.count() == data.drop_duplicates().count())
    print("There are No Duplicate Values")
except:
    print(f"There are Duplicate entires {data.count()} and {data.drop_duplicates().count()}")

There are Duplicate entires 12330 and 12205


In [13]:
from pyspark.sql import Window

partition_cols = data.columns
windowSpec = Window.partitionBy(partition_cols).orderBy('Administrative')

duplicateRecords = (data.withColumn('Row_Number', F.row_number().over(windowSpec))
    .filter(F.col('Row_Number') > 1))

duplicateRecords.show()

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+----------------+-------+------+-----------+-----------------+-------+-------------+-------------------+-----+----------+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|ExitRates|PageValues|SpecialDay|OperatingSystems|Browser|Region|TrafficType|      VisitorType|Weekend|made_purchase|ingestion_timestamp|Month|Row_Number|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+----------------+-------+------+-----------+-----------------+-------+-------------+-------------------+-----+----------+
|             0|                    0.0|            0|                   0.0|             1|                    0.0|        0.2|      0.2|       0.0|     

In [14]:
dataN = (data.withColumn('Row_Number', F.row_number().over(windowSpec))
            .filter(F.col("Row_Number") == 1))

try:
    assert(dataN.count() == data.drop_duplicates().count())
    print("There are No Duplicate Values")
except:
    print(f"There are Duplicate entires {dataN.count()} and {data.drop_duplicates().count()}")


There are No Duplicate Values


In [15]:
data = dataN.drop("Row_Number")

In [16]:
data.persist()

DataFrame[Administrative: int, Administrative_Duration: double, Informational: int, Informational_Duration: double, ProductRelated: int, ProductRelated_Duration: double, BounceRates: double, ExitRates: double, PageValues: double, SpecialDay: double, OperatingSystems: int, Browser: int, Region: int, TrafficType: int, VisitorType: string, Weekend: boolean, made_purchase: boolean, ingestion_timestamp: date, Month: string]

#### 7.	Flag rows where BounceRates > 1.

In [17]:
data.filter(F.col('BounceRates') > 1.).show()

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+----------------+-------+------+-----------+-----------+-------+-------------+-------------------+-----+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|ExitRates|PageValues|SpecialDay|OperatingSystems|Browser|Region|TrafficType|VisitorType|Weekend|made_purchase|ingestion_timestamp|Month|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+----------------+-------+------+-----------+-----------+-------+-------------+-------------------+-----+
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+----------------+-------+------+-----------+-

In [18]:
data.select('BounceRates').describe().show()

+-------+--------------------+
|summary|         BounceRates|
+-------+--------------------+
|  count|               12205|
|   mean|0.020370317181810746|
| stddev|0.045255441088393585|
|    min|                 0.0|
|    max|                 0.2|
+-------+--------------------+



#### 8. Count sessions with invalid ExitRates.

In [20]:
data.groupBy('VisitorType').agg(F.mean('ExitRates').alias('Avg_ExitRates')).show()

+-----------------+--------------------+
|      VisitorType|       Avg_ExitRates|
+-----------------+--------------------+
|      New_Visitor|0.020574603134081507|
|Returning_Visitor| 0.04473877444588245|
|            Other| 0.05660045814814814|
+-----------------+--------------------+



#### 9. Replace invalid numeric values with null.

In [21]:
numeric_col_names

['Administrative',
 'Administrative_Duration',
 'Informational',
 'Informational_Duration',
 'ProductRelated',
 'ProductRelated_Duration',
 'BounceRates',
 'ExitRates',
 'PageValues',
 'SpecialDay',
 'OperatingSystems',
 'Browser',
 'Region',
 'TrafficType']

#### 10. Create a data quality report DataFrame.

In [ ]:
data.describe().show()

DataFrame[summary: string, Administrative: string, Administrative_Duration: string, Informational: string, Informational_Duration: string, ProductRelated: string, ProductRelated_Duration: string, BounceRates: string, ExitRates: string, PageValues: string, SpecialDay: string, OperatingSystems: string, Browser: string, Region: string, TrafficType: string, VisitorType: string, Month: string]